In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.io import loadmat
from sklearn.metrics import classification_report#这个包是评价报告

In [2]:
data=loadmat('ex3data1.mat') #加载数据
data


{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
data['X'].shape,data['y'].shape

((5000, 400), (5000, 1))

In [4]:
# sigmoid函数
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [5]:
# 代价函数
def cost(theta,X,y,learningRate):
    theta=np.matrix(theta)
    X=np.matrix(X)
    y=np.matrix(y)

    cross_cost=np.multiply(-y,np.log(sigmoid(X*theta.T)))-np.multiply((1-y),np.log(1-sigmoid(X*theta.T)))

    reg=(learningRate/(2*len(X)))*np.sum(np.power(theta[1:],2))

    whole_cost=np.sum(cross_cost)/len(X)+reg
    return whole_cost

In [6]:
def gradient(theta,X,y,learningRate):
    theta=np.matrix(theta)
    X=np.matrix(X)
    y=np.matrix(y)

    parameters=int(theta.ravel().shape[1])

    error=sigmoid(X*theta.T)-y

    grad=((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)

    grad[0,0]=np.sum(np.multiply(error,X[:,0]))/len(X)

    return np.array(grad).ravel()

In [7]:
from scipy.optimize import minimize

def one_vs_all(X,y,num_labels,learning_rate):
    rows=X.shape[0]
    params=X.shape[1]

    all_theta=np.zeros((num_labels,params + 1))

    X = np.insert(X,0,values=np.ones(rows),axis=1)

    for i in range(1,num_labels+1):
        theta=np.zeros(params+1)
        y_i=np.array([1 if label==i else 0 for label in y])
        y_i=np.reshape(y_i,(rows,1))

        fmin=minimize(fun=cost,x0=theta,args=(X,y_i,learning_rate),method='TNC',jac=gradient)
        all_theta[i-1,:]=fmin.x

    return all_theta

In [8]:
rows=data['X'].shape[0]
params=data['X'].shape[1]

all_theta=np.zeros((10,params+1))
X=np.insert(data['X'],0,values=np.ones(rows),axis=1)

theta=np.zeros(params+1)
y_0=np.array([1 if label==0 else 0 for label in data['y']])
y_0=np.reshape(y_0,(rows,1))

X.shape,y_0.shape,theta.shape,all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [9]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [10]:
all_theta = one_vs_all(data['X'],data['y'],10,1)
all_theta


array([[-2.36695980e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30581285e-03, -1.09792571e-09,  0.00000000e+00],
       [-3.17878643e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.45176571e-03, -5.07488590e-04,  0.00000000e+00],
       [-4.78750938e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.85341155e-05, -2.48507893e-07,  0.00000000e+00],
       ...,
       [-7.97033240e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.00142591e-04,  7.82795440e-06,  0.00000000e+00],
       [-4.55826209e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.11572640e-03,  8.19315457e-05,  0.00000000e+00],
       [-5.30389282e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.13078635e-04,  9.26790970e-06,  0.00000000e+00]])

In [11]:
def predict_all(X,all_theta):
    rows=X.shape[0]
    params=X.shape[1]
    num_labels=all_theta.shape[0]

    X=np.insert(X,0,values=np.ones(rows),axis=1)
    X=np.matrix(X)
    all_theta=np.matrix(all_theta)

    h=sigmoid(X*all_theta.T)
    h_argmax=np.argmax(h,axis=1)
    h_argmax=h_argmax+1
    return h_argmax

In [12]:
y_pred=predict_all(data['X'],all_theta)
correct=[1 if a==b else 0 for (a,b) in zip(y_pred,data['y'])]
accuracy=(sum(map(int,correct))/float(len(correct)))
print('accuracy = {0}%'.format(accuracy*100))


accuracy = 94.48%
